### structured-output-parser
* LLM 나오는 답변을 dict 형식으로 구성해ㅈ준다.
* key/value 쌍으로 필드를 반환하고 싶을때 사용 할 수 있다.
* Gpt-4 또는 Calude 모델보다 파라미터수가 적은 로컬모델들 같은 경우에 효과적이다.


In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.output_parsers import ResponseSchema,StructuredOutputParser

response_schema = (
    ResponseSchema(name="answer",description="사용자 질문에 대한 답변"),
    ResponseSchema(
        name="source",
        description="사용자의 질문에 답하기 위해 사용된 '출처', '웹사이트주소' 이어야한다. "
    )
)

# 응답 스키마를 기반으로 StructuredOutputParser 생성
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

In [4]:
from langchain_core.prompts import PromptTemplate

format_instructions = output_parser.get_format_instructions()

print(format_instructions)

prompt = PromptTemplate(
    template="answer the question as beset as possible \n {format_instructions} \n Question\n {question}",
    input_variables=["question"],
    partial_variables={"format_instructions": format_instructions}
)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"answer": string  // 사용자 질문에 대한 답변
	"source": string  // 사용자의 질문에 답하기 위해 사용된 '출처', '웹사이트주소' 이어야한다. 
}
```


In [8]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(temperature=0,model="gpt-4o")
chain = prompt | model | output_parser

In [11]:
print(chain.invoke({"question":"Ollama 를 한글로 설명해줘"}))

{'answer': 'Ollama는 인공지능 및 머신러닝 모델을 개발하고 배포하는 플랫폼으로, 사용자가 쉽게 AI 모델을 구축하고 활용할 수 있도록 다양한 도구와 서비스를 제공합니다.', 'source': 'https://ollama.com'}
